In [1]:
!pip install transformers torch ipython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 48.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from IPython.display import display, HTML

In [3]:
repo = "nqzfaizal77ai/mt-230m-pretrained-en-1bc-exp"

In [4]:
precision = 'full' # @param ["half", "full"]
if precision == "half":
    selected_precision = torch.float16
elif precision == "full":
    selected_precision = torch.float32

In [5]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [6]:
model = AutoModelForCausalLM.from_pretrained(repo, torch_dtype=selected_precision)
model.to(device)
tokenizer = AutoTokenizer.from_pretrained(repo)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/465M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

In [7]:
# @title Hyperparameter Inference
max_length = 100  # @param {type:"integer"} (maximum length of generated text)
min_length = 10   # @param {type:"integer"} (minimum length should be less than max_length)
top_k = 50        # @param {type:"integer"} (typical range 0-100)
top_p = 0.92      # @param {type:"number"} (should be <= 1.0, typically 0.7-0.95)
temperature = 0.7  # @param {type:"number"} (typically 0.1-1.0, higher = more random)
repetition_penalty = 1.2  # @param {type:"number"} (typically 1.0-1.5)
do_sample = True   # @param {type:"boolean"}
use_cache = True   # @param {type:"boolean"}

In [19]:
prompt = "The quick brown fox jumps over the lazy dog"

Stochastic Decoding

In [20]:
inputs = tokenizer(prompt, return_tensors="pt").to(device)
output = model.generate(
                        inputs["input_ids"],
                        max_length=max_length,
                        min_length=min_length,
                        # num_return_sequences=1,
                        top_k=top_k,
                        top_p=top_p,
                        temperature=temperature,
                        repetition_penalty=repetition_penalty,
                        # num_beams=5,
                        # length_penalty=0.8,
                        # penalty_alpha=0.5,
                        do_sample=True,
                        use_cache=True
                        )
decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Greedy Decoding

In [22]:
# prompt: make greedy decoding code
inputs = tokenizer(prompt, return_tensors="pt").to(device)
# Greedy Decoding
greedy_output = model.generate(
    inputs["input_ids"],
    max_length=max_length,
    min_length=min_length,
    do_sample=False  # Set do_sample to False for greedy decoding
)
decoded_output = tokenizer.decode(greedy_output[0], skip_special_tokens=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [23]:
# HTML and CSS for displaying the output in a square box
html_code = f"""
<div style='border: 2px solid black; padding: 10px; width: 500px; height: 300px; overflow: auto;'>
    <p style='font-family: monospace;'>{decoded_output}</p>
</div>
"""

# Display the styled output
display(HTML(html_code))